# 03 - Interactive Viz

## Deadline
Friday October 28, 2016 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution
you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code

## Background
In this homework we will practice with interactive visualization, which is the key ingredient of many successful viz (especially when it comes to infographics).
You will be working with the P3 database of the [SNSF](http://www.snf.ch/en/Pages/default.aspx) (Swiss National Science Foundation).
As you can see from their [entry page](http://p3.snf.ch/), P3 already offers some ready-made viz, but we want to build a more advanced one for the sake
of quick data exploration. Therefore, start by [downloading the raw data](http://p3.snf.ch/Pages/DataAndDocumentation.aspx) (just for the Grant Export), and read carefully
the documentation to understand the schema. Install then [Folium](https://github.com/python-visualization/folium) to deal with geographical data (*HINT*: it is not
available in your standard Anaconda environment, therefore search on the Web how to install it easily!) The README file of Folium comes with very clear examples, and links 
to their own iPython Notebooks -- make good use of this information. For your own convenience, in this same directory you can already find a TopoJSON file with the 
geo-coordinates of each Swiss canton (which can be used as an overlay on the Folium maps).


## Assignment
1. Build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows intuitively (i.e., use colors wisely) how much grant money goes to each Swiss canton.
To do so, you will need to use the provided TopoJSON file, combined with the Choropleth map example you can find in the Folium README file.

*HINT*: the P3 database is formed by entries which assign a grant (and its approved amount) to a University name. Therefore you will need a smart strategy to go from University
to Canton name. The [Geonames Full Text Search API in JSON](http://www.geonames.org/export/web-services.html) can help you with this -- try to use it as much as possible
to build the canton mappings that you need. For those universities for which you cannot find a mapping via the API, you are then allowed to build it manually -- feel free to stop 
by the time you mapped the top-95% of the universities. I also recommend you to use an intermediate viz step for debugging purposes, showing all the universties as markers in your map (e.g., if you don't select the right results from the Geonames API, some of your markers might be placed on nearby countries...)

2. *BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in research funding
between the areas divided by the [Röstigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?

*HINT*: for those cantons cut through by the Röstigraben, [this viz](http://p3.snf.ch/Default.aspx?id=allcharts) can be helpful!


In [ ]:
from bs4 import BeautifulSoup
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
sns.set_context('notebook')
import re
import requests as rq
import json
import google_helper as ggl_h

In [ ]:
cantons_json_path = "data/ch-cantons.topojson.json"
cantons_csv_path = "data/P3_GrantExport.csv"

data = pd.read_csv(cantons_csv_path, sep=';')

### Drop columns we don't need

In [ ]:
data = data.drop([ "Project Title", "Project Title English", "Responsible Applicant", "Funding Instrument", "Funding Instrument Hierarchy", "Discipline Number", "Discipline Name", "Discipline Name Hierarchy", "Start Date", "End Date", "Keywords"], axis=1)

In [ ]:
data.head()

### Drop lines where amount is null and where University is not declared

We drop the line where the amount column equals "Data not included in P" as it is not interessant to keep it if we need to compute the sum of the amounts.

Then we drop the lines where the university column equals "Nicht zuteilbar - NA" and "NA" as we cannot find the cantons where they are placed.

In [ ]:
count_amount = data[data["Approved Amount"]=="data not included in P3"].size
count_uni = data[data["University"]=="Nicht zuteilbar - NA"].size + data[data["University"]=="NaN"].size
print("Percentage of amount data not in P3 : ",count_amount/data.size*100,"%")
print("Percentage of University not defined : ",count_uni/data.size*100,"%")
print("Total number of lines in the data before deletion : ",data.size)
print("Number of lines deleted : ",count_amount + count_uni)
print("Total percentage of lines deleted : ", (count_amount+count_uni)/data.size*100, "%")
data = data[data["Approved Amount"]!="data not included in P3"]
data = data[data["University"]!="Nicht zuteilbar - NA"]
data = data.dropna(subset=['University'])
print(data.shape)
print(data.shape)
data.head()

It deleted 54020 lines, which is around 21% of the lines in the given data. It is a lot but necessary.

#### We reset the index to keep an index going from 0 to the length of the table

In [ ]:
data = data.reset_index(drop=True)
data.head()

#### Find unique university names

In [ ]:
universities = data.University.unique()
universities[:5]

### How to find the cantons for each university

First we remarked that some universities are described by their Canton (ex : "Université de Genève - GE"). For those ones we want to just get the Canton code and add it in the correct column.

Then some others are not described by their Canton so in this case we use GeoNames

### Extract canton's code directly from University names given in data

First of all, we need to know which code describe a canton, we can find this in the json

In [ ]:
"""
# This gets the value after the '-' in the uni name :
# re.findall(r'- (\w+)', data['University'][index])[0]
input_file=open(cantons_json_path, 'r', encoding='utf-8')
json_decode=json.load(input_file)
canton_ids = []
canton_names = []
for i in range(0,len(json_decode['objects']['cantons']['geometries'])):
    canton_ids.append(json_decode['objects']['cantons']['geometries'][i]['id'])
    canton_names.append(json_decode['objects']['cantons']['geometries'][i]['properties']['name'])
print(canton_ids)
print(canton_names)
"""

For cantons with two names (French/German/Italian), let's separate them

In [ ]:
"""
canton_names_sep = []
for canton_name in canton_names:
    if('/' in canton_name):
        canton_names_sep.append(re.findall(r'(\w+)/', canton_name)[0])
        canton_names_sep.append(re.findall(r'/(\w+)', canton_name)[0])
    else:
        canton_names_sep.append(canton_name)
"""

Now we want to go through the data and we look if in the name of the university there is an occurance of one of the names and we assume they are from this canton
First, we create the dataframe

### Use Google API to find cantons of University

In [ ]:
cantons_ids_google = ggl_h.canton_ids(universities)

#cantons_ids = ggl_h.canton_ids(['Université de Neuchâtel'])
cantons_ids_google

### Create new dataframe with amount and cantons

In [ ]:
input_file = open(cantons_json_path, 'r', encoding='utf-8')
json_decode = json.load(input_file)
canton_ids = []
canton_names = []
for i in range(0, len(json_decode['objects']['cantons']['geometries'])):
    canton_ids.append(json_decode['objects']['cantons']['geometries'][i]['id'])
 

correct     = [t for t in cantons_ids_google if t[1] in canton_ids]
not_correct = [t for t in cantons_ids_google if t[1] not in canton_ids]


In [ ]:
#Build lookup manually
lookup = {
 'AO Research Institute': 'GR',
 'Allergie': 'BE',
 'Eidg. Forschungsanstalt für Wald,Schnee,Land': 'ZH',
 'Eidg. Material und Prüfungsanstalt': 'ZH',
 'Fachhochschule Nordwestschweiz (ohne PH)': 'SO',
 'Firmen/Privatwirtschaft': None, #private society/sector so no location available
 'Forschungsanstalten Agroscope': None, #organism split into multiple cantons., can't assume equal distribution
 'Forschungsinstitut für Opthalmologie': 'VS',
 'Forschungsinstitut für biologischen Landbau': 'AG',
 'Forschungskommission SAGW': 'BE',
 'Haute école pédagogique BE, JU, NE': 'JU',
 'Inst. de Hautes Etudes Internat. et du Dév': 'GE',
 'Istituto Svizzero di Roma': None, #private fondation, in Rome
 'NPO (Biblioth., Museen, Verwalt.)': None, #non-profit organizations, can't assume equal distribution
 'Physikal.': None, #swiss physical society, can't assume equal distribution
 'Schweiz. Hochschule für Logopädie Rorschach': 'SG',
 'Schweizer Kompetenzzentrum Sozialwissensch.': 'VD',
 'Swiss Center for Electronics and Microtech.': 'NE',
 'Swiss Institute of Bioinformatics': 'VD',
 'Weitere Institute': None, #other institutes, can't assume equals distribution
 'Weitere Spitäler': None #other hospitals, can't assume equal distribution
}

In [ ]:
map_ids = {**dict(correct), **lookup}

In [ ]:
data_bis = data.copy()
data_bis['Canton'] = data_bis['University'].apply(lambda name: map_ids[name.split('-')[0].strip()])
data_bis = data_bis[['Canton', 'Approved Amount']]
data_bis['Approved Amount'] = data_bis['Approved Amount'].apply(lambda val : float(val))
data_bis = data_bis.groupby('Canton').sum()
data_bis = data_bis.reset_index()
cantons_missing = [[id_, 0] for id_ in canton_ids if id_ not in data_bis.Canton.values]
df_missing = pd.DataFrame(cantons_missing, columns=['Canton', 'Approved Amount'])
data_bis = data_bis.append(df_missing, ignore_index=True)
#data_bis['Approved Amount'] = data_bis['Approved Amount'].apply(lambda gr: (int)(gr))
thresh = list(np.linspace(data_bis['Approved Amount'].min() + 1, data_bis['Approved Amount'].max(), 6, dtype=np.int))

In [ ]:
cantons_coordinates = {
    'ZH' : [47.377895, 8.541183],
    'BE' : [46.94909, 7.447357], 
    'LU' : [47.05, 8.3], 
    'UR' : [46.880826, 8.639446], 
    'SZ' : [47.020546, 8.658332], 
    'OW' : [46.897214, 8.24722], 
    'NW' : [46.95972, 8.366674], 
    'GL' : [47.033329, 9.066666], 
    'ZG' : [47.169441, 8.516663], 
    'FR' : [46.806103, 7.162775], 
    'SO' : [47.208331, 7.537513], 
    'BS' : [47.567, 7.583], 
    'BL' : [47.466667, 7.733333], 
    'SH' : [47.700001, 8.633333], 
    'AR' : [47.383329, 9.266671], 
    'AI' : [47.330828, 9.408615], 
    'SG' : [47.416667, 9.366667], 
    'GR' : [46.85, 9.533333],
    'AG' : [47.4, 8.05], 
    'TG' : [47.556, 8.8965], 
    'TI' : [46.2, 9.016667], 
    'VD' : [46.521, 6.631], 
    'VS' : [46.227778, 7.358611], 
    'NE' : [46.990281, 6.930567], 
    'GE' : [46.200013, 6.149985], 
    'JU' : [47.366667, 7.35]
}

In [ ]:
# TEST
import locale 

locale.setlocale(locale.LC_ALL, '')

swiss_map = folium.Map(location=[47, 8], zoom_start=8)
swiss_map.choropleth(geo_path=cantons_json_path, 
                     data=data_bis,
                     columns=['Canton', 'Approved Amount'],
                     threshold_scale=thresh,
                     key_on='feature.id',
                     fill_opacity = 1,
                     topojson='objects.cantons',
                     fill_color='OrRd',
                     legend_name = 'Grant money for each Swiss canton'
                    )

for key, value in cantons_coordinates.items():
    if(value != None):
        amount = locale.currency(data_bis[data_bis['Canton'] == key]['Approved Amount'].values[0], grouping=True)
        folium.RegularPolygonMarker(
                   location=value, 
                   popup=key + ' - ' + amount,
                   fill_color='#43d9de', 
                   number_of_sides=4, 
                   radius=10).add_to(swiss_map)
        
swiss_map.save('swiss_map.html')